# Skip-gram Word2Vec

In this notebook, I'll lead you through using PyTorch to implement the [Word2Vec algorithm](https://en.wikipedia.org/wiki/Word2vec) using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of Word2Vec from Chris McCormick 
* [First Word2Vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [Neural Information Processing Systems, paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for Word2Vec also from Mikolov et al.

---
## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of word classes to analyze; one for each word in a vocabulary. Trying to one-hot encode these words is massively inefficient because most values in a one-hot vector will be set to zero. So, the matrix multiplication that happens in between a one-hot input vector and a first, hidden layer will result in mostly zero-valued hidden outputs.

<img src='assets/one_hot_encoding.png' width=50%>

To solve this problem and greatly increase the efficiency of our networks, we use what are called **embeddings**. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

<img src='assets/lookup_matrix.png' width=50%>

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=50%>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.

---
## Word2Vec

The Word2Vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words.

<img src="assets/context_drink.png" width=40%>

Words that show up in similar **contexts**, such as "coffee", "tea", and "water" will have vectors near each other. Different words will be further away from one another, and relationships can be represented by distance in vector space.

<img src="assets/vector_distance.png" width=40%>


There are two architectures for implementing Word2Vec:
>* CBOW (Continuous Bag-Of-Words) and 
* Skip-gram

<img src="assets/word2vec_architectures.png" width=60%>

In this implementation, we'll be using the **skip-gram architecture** because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

---
## Loading Data

Next, we'll ask you to load in data and place it in the `data` directory

1. Load the [text8 dataset](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip); a file of cleaned up *Wikipedia article text* from Matt Mahoney. 
2. Place that data in the `data` folder in the home directory.
3. Then you can extract it and delete the archive, zip file to save storage space.

After following these steps, you should have one file in your data directory: `data/text8`.

In [1]:
# read in the extracted text file      
with open('data/text8') as f:
    text = f.read()

# print out the first 100 characters
print(text[:100])

 anarchism originated as a term of abuse first used against early working class radicals including t


## Pre-processing

Here I'm fixing up the text to make training easier. This comes from the `utils.py` file. The `preprocess` function does a few things:
>* It converts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. 
* It removes all words that show up five or *fewer* times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. 
* It returns a list of words in the text.

This may take a few seconds to run, since our text file is quite large. If you want to write your own functions for this stuff, go for it!

In [2]:
import utils

# get list of words
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [3]:
# print some stats about this word data
print("Total words in text: {}".format(len(words)))
print("Unique words: {}".format(len(set(words)))) # `set` removes any duplicate words

Total words in text: 16680599
Unique words: 63641


### Dictionaries

Next, I'm creating two dictionaries to convert words to integers and back again (integers to words). This is again done with a function in the `utils.py` file. `create_lookup_tables` takes in a list of words in a text and returns two dictionaries.
>* The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1, and so on. 

Once we have our dictionaries, the words are converted to integers and stored in the list `int_words`.

In [4]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

[5233, 3080, 11, 5, 194, 1, 3133, 45, 58, 155, 127, 741, 476, 10571, 133, 0, 27349, 1, 0, 102, 854, 2, 0, 15067, 58112, 1, 0, 150, 854, 3580]


## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

$$ P(0) = 1 - \sqrt{\frac{1*10^{-5}}{1*10^6/16*10^6}} = 0.98735 $$

I'm going to leave this up to you as an exercise. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [5]:
from collections import Counter
import random
import numpy as np

threshold = 1e-5
word_counts = Counter(int_words)
print(list(word_counts.items())[0])  # dictionary of int_words, how many times they appear

# discard some frequent words, according to the subsampling equation
# create a new list of words for training
total_count = len(int_words)
probabilities = 1 - np.sqrt(threshold / (np.array(list(word_counts.values())) / total_count))
probability_lookup = { word: prob for word, prob in zip(word_counts.keys(), probabilities) }
train_words = [word for word in int_words if random.random() > probability_lookup[word]]

print("Before: ", len(int_words))
print("After:", len(train_words))
print(train_words[:30])

(5233, 303)
Before:  16680599
After: 4628446
[5233, 1, 3133, 45, 476, 10571, 27349, 15067, 58112, 854, 58, 10712, 1324, 454, 58, 2731, 3672, 7088, 5233, 1052, 44611, 2877, 792, 200, 2621, 8983, 4147, 6437, 1, 5233]


## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to define a surrounding _context_ and grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $[ 1: C ]$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you chose a random number of words to from the window.

Say, we have an input and we're interested in the idx=2 token, `741`: 
```
[5233, 58, 741, 10571, 27349, 0, 15067, 58112, 3580, 58, 10712]
```

For `R=2`, `get_target` should return a list of four values:
```
[5233, 58, 10571, 27349]
```

In [6]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''

    word_range = random.randint(1, window_size)
    
    return words[max(0, idx - word_range):idx] + words[idx + 1:min(len(words), idx + 1 + word_range)]

In [7]:
# test your code!

# run this cell multiple times to check for random window selection
int_text = [i for i in range(10)]
print('Input: ', int_text)
idx=5 # word index of interest

target = get_target(int_text, idx=idx, window_size=5)
print('Target: ', target)  # you should get some indices around the idx

Input:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Target:  [4, 6]


### Generating Batches 

Here's a generator function that returns batches of input and target data for our model, using the `get_target` function from above. The idea is that it grabs `batch_size` words from a words list. Then for each of those batches, it gets the target words in a window.

In [8]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

In [9]:
int_text = [i for i in range(20)]
x,y = next(get_batches(int_text, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

x
 [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3]
y
 [1, 2, 3, 0, 2, 3, 0, 1, 3, 0, 1, 2]


## Building the graph

Below is an approximate diagram of the general structure of our network.
<img src="assets/skip_gram_arch.png" width=60%>

>* The input words are passed in as batches of input word tokens. 
* This will go into a hidden layer of linear units (our embedding layer). 
* Then, finally into a softmax output layer. 

We'll use the softmax layer to make a prediction about the context words by sampling, as usual.

The idea here is to train the embedding layer weight matrix to find efficient representations for our words. We can discard the softmax layer because we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in _other_ networks we build using this dataset.

---
## Validation

Here, I'm creating a function that will help us observe our model as it learns. We're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them using the cosine similarity: 

<img src="assets/two_vectors.png" width=30%>

$$
\mathrm{similarity} = \cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|}
$$


We can encode the validation words as vectors $\vec{a}$ using the embedding table, then calculate the similarity with each word vector $\vec{b}$ in the embedding table. With the similarities, we can print out the validation words and words in our embedding table semantically similar to those words. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [10]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # pick N words from our ranges (0,window) and (1000,1000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

## SkipGram model

Define and train the SkipGram model. 
> You'll need to define an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) and a final, softmax output layer.

An Embedding layer takes in a number of inputs, importantly:
* **num_embeddings** – the size of the dictionary of embeddings, or how many rows you'll want in the embedding weight matrix
* **embedding_dim** – the size of each embedding vector; the embedding dimension

In [11]:
import torch
from torch import nn
import torch.optim as optim

In [12]:
class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        
        # complete this SkipGram model
        self.embed = nn.Embedding(n_vocab, n_embed)
        self.fc1 = nn.Linear(n_embed, n_vocab)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        return self.softmax(self.fc1(self.embed(x)))

### Training

Below is our training loop, and I recommend that you train on GPU, if available.

**Note that, because we applied a softmax function to our model output, we are using NLLLoss** as opposed to cross entropy. This is because Softmax  in combination with NLLLoss = CrossEntropy loss .

In [15]:
# check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Training on: {device}")

embedding_dim=300 # you can change, if you want

model = SkipGram(len(vocab_to_int), embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 500
steps = 0
epochs = 5

# train for some number of epochs
for e in range(epochs):
    print(f"Epoch {e + 1}/{epochs}")
    
    # get input and target batches
    for inputs, targets in get_batches(train_words, 32):
        steps += 1
        inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if steps % print_every == 0:                  
            # getting examples and similarities      
            valid_examples, valid_similarities = cosine_similarity(model.embed, device=device)
            _, closest_idxs = valid_similarities.topk(6) # topk highest similarities
            
            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...")

Training on: cuda
two | clandestinely, calico, sumerian, cajuns, tossed
a | masterwork, boziz, favourable, gent, theta
see | mitsubishi, freire, mu, archaeology, eidyn
would | uruguayan, lancelot, quantum, fungus, xserve
zero | softpanorama, pubns, contagious, petersen, joyous
s | produce, georgy, jawed, anarchists, ascended
all | ong, whiz, southsea, homegrown, evidential
not | monosaccharides, ahijah, forbade, ikk, marcian
accepted | doonesbury, scolded, muster, esteban, diazepam
orthodox | usefully, aerobics, drowned, relinquished, ns
police | rtiges, nationalized, rhenium, antacids, escapes
event | ambracia, guelph, sicily, sampa, depressing
joseph | noin, beating, imperatoribus, genova, unsubstantiated
creation | qui, chinese, chemists, armalite, integrates
operations | wedding, shouting, filipino, mascots, indic
shown | novelas, brass, ripken, pinched, rays
...
other | haben, easel, desserts, arghun, judgment
can | sloop, trichloride, misfortunes, twigs, perennially
three | layer

often | overextended, normalisation, franklin, cumings, gru
zero | joyous, softpanorama, contagious, nanking, glace
six | pion, ranked, settlements, hira, fishman
his | obtain, sliced, aznar, brotherhoods, pusc
can | sloop, pulp, questionnaire, misfortunes, duncan
nine | larval, ikeda, corinthian, claudia, castille
over | mining, carradine, cromwell, gower, stockhausen
that | archaeologist, warranted, proverbs, obama, reporters
grand | selma, classless, vacancies, admiral, ensure
recorded | sqrta, lf, waterways, moc, mgn
resources | polycarp, stevens, glandular, colloquialisms, coinages
shown | novelas, rays, brass, ripken, pitfall
dr | languished, lactose, truthfulness, incarnate, tcm
accepted | doonesbury, scolded, muster, teach, diazepam
stage | sears, erin, landsmannschaft, heartbreak, leonov
event | ambracia, severe, guelph, depressing, sicily
...
a | boziz, masterwork, tangenziale, theta, favourable
eight | versailles, crescas, surfing, noxious, dancers
or | quart, mafia, croplan

their | nouns, essences, bounced, necromancy, mong
or | quart, mafia, cropland, mchugh, fork
than | okresy, zwolle, searchable, peg, constantly
known | digraph, democratic, wings, cubits, passim
also | personage, niv, slackware, layla, flamboyant
however | suspicion, resentments, gesture, forwarded, amistad
are | daniell, theists, religiosity, hatfield, marilyn
used | egotistical, pavilion, bw, chrysler, harrison
mean | tackle, tampering, studd, insignificant, contended
defense | ayla, milford, dalmatia, stockwell, predict
powers | imitation, caltech, misfits, ingersoll, eithne
pre | newstead, mms, allotropic, forbidden, commentary
event | ambracia, guelph, severe, sampled, draper
versions | vint, zwickau, arkham, aonb, turbines
paris | meatball, jolley, philoctetes, discussing, creasy
ocean | admission, map, wav, willamette, bunghole
...
in | weary, user, dmitry, misanthropy, nocturne
over | demonstration, nasl, conciliatory, stockhausen, carradine
that | warranted, auction, cept, pro

its | timespan, bb, mansfield, urbino, carom
two | zero, nine, one, american, gregor
i | razed, literature, twos, anthropologist, concealing
been | freer, splines, seizing, holberg, chi
zero | nine, two, one, eight, four
seven | one, six, nine, four, three
there | swordsmanship, virial, capturing, preface, idolatrous
while | gacy, withdrawal, pahlavi, college, tobruk
applied | website, chronicling, wealthier, distract, causality
articles | kite, judeo, dishonest, domitius, daybreak
powers | imitation, eithne, caltech, enable, ingersoll
behind | shirelles, participant, mcnamara, balhae, richelieu
file | expandability, blissymbols, homeowners, limes, hooft
joseph | unsubstantiated, noin, faso, genova, borrelly
bible | sisak, picket, striving, disagreement, kindia
lived | tedious, conservatism, cosmonauts, interludes, hepatitis
...
people | seperated, unsound, collimated, bestial, shakespeare
than | okresy, zwolle, lok, frictional, flared
had | pious, luisa, reverting, autobahn, new
many 

s | one, the, eight, of, on
can | kang, ugaritic, deployments, aino, hypertrophic
system | mcvie, vehicle, khanty, adelphi, appointments
one | eight, two, three, five, nine
four | eight, three, two, seven, zero
such | lutes, phagspa, nikolaevich, metal, joliet
seven | nine, four, zero, one, eight
from | and, gwent, mixture, khanate, derisively
magazine | genuineness, leo, solstices, joanna, sculptures
discovered | degas, recasting, rampant, nonspecific, gw
mainly | farragut, african, celera, browed, visionaries
construction | tl, coulsdon, yamamoto, poststructuralism, deepwater
ocean | admission, willamette, islamiyah, crux, wav
proposed | orf, logbook, kremvax, martyrology, qaddafi
brother | icarus, meigs, lycaeus, sackville, reservists
grand | admiral, november, selma, packers, harmonious
...
united | minneapolis, warlords, of, akbar, jettison
but | bicoid, disliked, hulme, equiv, subsonic
s | one, of, for, eight, the
eight | one, four, two, six, nine
for | the, a, s, in, of
about | 

...

however | of, the, to, in, at
six | eight, five, of, one, four
which | the, of, with, and, as
there | of, and, in, the, as
they | of, in, the, to, and
united | of, and, as, after, a
have | to, of, or, for, will
while | of, the, to, a, in
mainly | of, as, for, still, the
older | three, for, the, differentiate, fish
test | of, and, two, the, or
engineering | research, the, at, of, general
road | off, and, adjacent, street, vehicle
running | the, of, half, would, systems
operations | important, used, data, allow, iraq
numerous | and, of, with, an, as
...
also | a, be, the, as, to
who | an, a, be, and, by
the | of, and, in, a, as
a | and, of, the, with, to
more | the, are, a, an, and
this | the, of, to, in, any
which | the, with, a, are, of
has | of, are, these, no, a
construction | structure, the, on, british, as
mathematics | students, a, an, is, two
police | in, personnel, camp, national, combat
applications | computer, data, uses, high, server
lived | no, until, to, he, the
question | 

system | in, the, to, with, on
with | a, and, the, in, from
it | a, is, and, that, with
can | is, and, be, to, in
known | and, in, a, is, the
people | on, and, of, many, that
united | of, kingdom, and, a, states
three | four, zero, five, two, seven
mean | or, a, is, that, some
writers | authors, century, htm, novelists, and
issue | to, zero, now, however, related
marriage | divorce, son, sexual, called, married
bbc | original, television, tv, zero, on
alternative | for, original, the, is, appears
running | a, is, long, foundation, previous
cost | the, and, original, declined, less
...
world | the, and, not, many, in
new | the, a, over, february, two
american | and, from, a, as, also
only | a, to, as, and, not
in | and, a, are, the, to
has | and, to, a, in, the
may | is, to, a, not, an
all | the, as, and, a, is
ice | hockey, a, moves, players, the
experience | a, the, and, as, theory
behind | head, the, to, over, used
taking | the, many, and, is, their
instance | and, by, for, not, or
g

after | the, one, of, two, zero
often | the, a, to, by, of
d | b, eight, one, seven, three
history | of, the, zero, two, and
such | of, and, with, the, that
has | the, to, in, two, and
some | of, a, in, to, the
five | seven, one, four, zero, six
mainly | areas, of, east, to, a
derived | vowel, a, idea, consonant, with
orthodox | faith, christianity, church, judaism, countries
magazine | stories, a, of, to, seven
except | all, an, a, to, the
prince | eight, one, eldest, two, seven
woman | nationality, a, women, her, men
construction | of, in, for, near, the
...
so | and, the, as, by, i
two | one, six, and, zero, eight
to | the, first, in, he, and
history | and, two, zero, of, the
with | the, in, of, s, a
five | seven, four, one, six, two
may | seven, and, five, this, as
into | and, the, a, in, of
event | and, two, that, before, indonesia
cost | as, even, of, s, and
recorded | six, by, for, the, one
institute | research, doctorate, state, isbn, humanities
question | asked, logical, in, i

these | a, to, and, the, in
where | and, which, a, as, of
all | and, the, in, of, a
who | and, for, the, his, in
eight | two, nine, zero, four, the
most | and, the, of, a, as
such | an, with, for, and, that
is | as, and, s, a, the
road | and, plans, part, for, began
lived | s, and, early, first, former
hold | as, states, which, however, a
http | html, www, edu, two, htm
mainly | and, part, however, to, but
discovered | and, of, for, the, s
units | unit, prefix, and, all, that
derived | and, s, by, it, unit
...
called | is, the, have, and, a
a | the, and, to, of, from
it | a, as, is, the, to
of | the, and, to, for, a
have | and, that, as, with, is
up | is, are, a, for, brown
zero | two, nine, one, seven, eight
new | by, in, the, and, of
behind | the, with, in, for, on
instance | that, for, grammatical, in, are
professional | has, however, five, have, football
mathematics | has, set, as, hilbert, student
pressure | and, when, the, into, a
prince | home, duke, who, stewart, on
articles | 

KeyboardInterrupt: 

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [18]:
torch.save(model, "embed.pth")

In [13]:
model = torch.load("embed.pth")

In [14]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [15]:
# getting embeddings from the embedding layer of our model, by name
embeddings = model.embed.weight.to('cpu').data.numpy()

In [16]:
viz_words = 600
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])

C:\Users\Creator\anaconda3\envs\udacity\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\Creator\anaconda3\envs\udacity\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [17]:
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)